In [10]:
import statistics
import collections
import getopt, sys
import matplotlib.pyplot as plt
import numpy as np
from numpy import linspace
import pandas
import math
from sklearn.preprocessing import StandardScaler
import ROOT
import root_numpy
from ROOT import gSystem
from root_numpy import root2array, array2tree
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import plot_model
from math import sqrt
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [11]:
class MyDict(collections.OrderedDict):
    def __missing__(self, key):
        val = self[key] = MyDict()
        return val

In [17]:
inTreeName = "preselection"
fileListLocation = "../datasets_to_append/"
nnLocation = "/home/paredes/pheno/testnn/pheno_study/analysis/neuralnet/train_all_jets"
inFileList = "test_append.txt"
#for testing only
analysis = "boosted"

In [18]:
branchList = ["pT_hh",
              "nMuon", "nElec",
              "h1_M", "h1_Pt", "h1_Eta", "h1_Phi", "h1_j1_j2_dR",
              "h2_M", "h2_Pt", "h2_Eta", "h2_Phi", "h2_j1_j2_dR",
              "h1_j1_M", "h1_j1_Pt", "h1_j1_Eta", "h1_j1_Phi",
              "h1_j2_M", "h1_j2_Pt", "h1_j2_Eta", "h1_j2_Phi",
              "h2_j1_M", "h2_j1_Pt", "h2_j1_Eta", "h2_j1_Phi",
              "h2_j2_M", "h2_j2_Pt", "h2_j2_Eta", "h2_j2_Phi",
              "muon1_M", "muon1_Pt", "muon1_Eta", "muon1_Phi",
              "elec1_M", "elec1_Pt", "elec1_Eta", "elec1_Phi",
              "met_Et", "met_Phi",
              "h1_j1_BTag","h1_j2_BTag","h2_j1_BTag","h2_j2_BTag",
              "mc_sf"]


In [29]:
f = open(inFileList,"r")
for line in f:
    # create a dictionary to store scores for all networks and event types
    scoreBranch = MyDict()
 
    inFilepath = line.rstrip()
    # Load the input file and grab the data from it.
    inArray = root2array(inFilepath, branches=branchList, treename=inTreeName)
    inFile = ROOT.TFile.Open(inFilepath)
    inTree = inFile.Get(inTreeName)
    print("Processing", inFilepath, len(inArray), "events found")
    nnFilepath = nnLocation + "/nn_all_jets_noTags_boostedall_jets_LR5e-05.h5"
    scalerFilepath = nnLocation + "/scaler_boostedall_jets_LR5e-05.sav"


Processing /home/paredes/pheno/testnn/pheno_study/utilities/SplitROOT/OxHHPh_06May2019_MG5_262_Py8_14TeV_NNPDF30NLO_Dlph3_pp2hh_4b_HeavyHiggsTHDM_TopYuk_1.0_SlfCoup_1.0_merged_ntup.root.boosted_0.root 12119 events found


In [30]:
    # Silly hack to remove stuctured dtype
    inDF = pandas.DataFrame(inArray)
    run_sample_sf = inDF.pop("mc_sf")
    inArray = inDF.values


In [35]:
    
    # Scale input features to mean=0, stddev=1
    scaler = joblib.load(scalerFilepath)
    inArray = scaler.fit_transform(inArray)
    model = keras.models.load_model(nnFilepath)


In [38]:
    scores = model.predict(inArray,batch_size=None)

In [40]:
scores

array([[0.23598132, 0.6636431 , 0.10037559],
       [0.48547456, 0.19615498, 0.31837043],
       [0.17034453, 0.7697709 , 0.05988456],
       ...,
       [0.03088052, 0.96418047, 0.00493904],
       [0.59658056, 0.0399632 , 0.3634562 ],
       [0.3124555 , 0.67703336, 0.0105111 ]], dtype=float32)